The lattice Wess-Zumino Hamiltonian is given by 

$$
  H = \sum_n \left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
             \quad \left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right) + \frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right], 
$$

For conceptual and code clarity we break this into four terms to specify contributions for interactions(I), non-interactions(NI) of terms with purely bosonic(B) or fermionic(F)/mixed terms.

$$
  H^B_{NI}=\frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2 \\ 
  H^B_{I}=\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \\
  H^F_{NI}=\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \\ 
  H^F_{I}=(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
$$

In [2]:
import numpy as np
from scipy.sparse import diags, kron, identity

In [3]:
N = 2 #sites
a = 1 #lattice spacing
m = 1 #mass

$$
\frac{p^2}{2a} = -\frac{\hbar^2}{2a} \frac{\partial^2}{\partial x^2}.
$$

Disrectized second derivative:

$$
\frac{\partial^2 \psi}{\partial x^2} \Big|_{x_n} \approx \frac{\psi_{n+1} - 2\psi_n + \psi_{n-1}}{a^2}.
$$

Final kinetic term matrix

$$
T =
\frac{1}{2a^3}
\begin{bmatrix}
  2 & -1 &  0 & \cdots & 0 \\
 -1 &  2 & -1 & \cdots & 0 \\
  0 & -1 &  2 & \cdots & 0 \\
  \vdots & \vdots & \vdots & \ddots & -1 \\
  0 &  0 &  0 & -1 & 2
\end{bmatrix}.

$$

In [12]:
# Bosonic kinetic energy:

kinetic_diag = -2 * np.ones(N)
off_diag = np.ones(N - 1)
T_b = diags([kinetic_diag, off_diag, off_diag], [0, -1, 1], format="csr") / 2*a**3
T_b.toarray()

array([[-1. ,  0.5],
       [ 0.5, -1. ]])

$$
\text{Original Term:}
\quad
\frac{a}{2} \left( \frac{\phi_{n+1} - \phi_{n-1}}{2a} \right)^2
\\
\text{Finite Difference Approximation:}
\quad
\frac{\phi_{n+1} - \phi_{n-1}}{2a} \approx \frac{\partial \phi}{\partial x}
\\
\text{Square of the Derivative:}
\quad
\left( \frac{\phi_{n+1} - \phi_{n-1}}{2a} \right)^2 = \frac{1}{4a^2} \left( \phi_{n+1}^2 - 2\phi_{n+1}\phi_{n-1} + \phi_{n-1}^2 \right)
\\
\text{Including the Prefactor:}
\quad
\frac{a}{2} \left( \frac{\phi_{n+1} - \phi_{n-1}}{2a} \right)^2 = \frac{1}{8a} \left( \phi_{n+1}^2 - 2\phi_{n+1}\phi_{n-1} + \phi_{n-1}^2 \right)
\\
\quad
\text{Diagonal: } \frac{\phi_{n+1}^2 + \phi_{n-1}^2}{8a}, \quad
\\
\text{Off-Diagonal: } -\frac{\phi_{n+1}\phi_{n-1}}{4a}.
\\
H = 
\begin{bmatrix}
\phi_1^2 & -\phi_1\phi_2 & 0 & \cdots & 0 \\
-\phi_2\phi_1 & \phi_2^2 & -\phi_2\phi_3 & \cdots & 0 \\
0 & -\phi_3\phi_2 & \phi_3^2 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & \phi_N^2
\end{bmatrix}.
$$

In [25]:
off_diag = np.ones(N - 1)  # Off-diagonal interactions
interaction_term = diags([off_diag, -off_diag], [-1, 1], format="csr") / (4 * a)
interaction_term.toarray()

array([[ 0.  , -0.25],
       [ 0.25,  0.  ]])

In [1]:
def create_wess_zumino_hamiltonian(potential_func, potential_derivative, dx, N):
    """
    Creates the Wess-Zumino Hamiltonian based on the given form.

    Parameters:
    - potential_func: function
        The potential function V(phi).
    - potential_derivative: function
        The derivative of the potential function, V'(phi).
    - dx: float
        Spacing between points in the discretized grid.
    - N: int
        Number of discretization points.

    Returns:
    - H_wz: ndarray
        The Hamiltonian matrix.
    """
    # Discretized grid
    phi_vals = np.linspace(-N // 2, N // 2, N) * dx

    # Bosonic kinetic energy: (p_n^2 / 2a)
    kinetic_diag = -2 * np.ones(N)
    off_diag = np.ones(N - 1)
    T_b = diags([kinetic_diag, off_diag, off_diag], [0, -1, 1], format="csr") / dx**2
    T_b *= 1 / (2 * dx)

    # Interaction term: (a/2) * [(phi_{n+1} - phi_{n-1}) / (2a)]^2
    interaction_term = diags(
        [np.ones(N - 1), -np.ones(N - 1)], [-1, 1], format="csr"
    ) / (4 * dx**2)

    # Bosonic potential terms
    V_phi = potential_func(phi_vals)
    V_b = 0.5 * V_phi**2
    V_coupling = V_phi * (diags([1, -1], [-1, 1], format="csr") @ phi_vals) / (2 * dx)

    V_b_matrix = diags(V_b, format="csr") + V_coupling

    # Fermionic terms
    staggered_signs = np.array([(-1)**i for i in range(N)])
    V_fermion = staggered_signs * potential_derivative(phi_vals)
    fermion_potential_matrix = diags(V_fermion, format="csr")

    # Fermionic hopping: (1 / 2a) (chi_n^† chi_{n+1} + chi_{n+1}^† chi_n)
    hopping_matrix = diags([np.ones(N - 1), np.ones(N - 1)], [-1, 1], format="csr") / (
        2 * dx
    )

    # Combine bosonic and fermionic terms
    I_b = identity(N)  # Identity for bosonic space
    I_f = np.eye(2)  # Identity for fermionic space

    H_b = T_b + interaction_term + V_b_matrix  # Total bosonic part
    H_f = fermion_potential_matrix + hopping_matrix  # Total fermionic part

    # Combine into a single Hamiltonian
    H_wz = kron(I_f, H_b) + kron(I_b, H_f)

    return H_wz.toarray()


# Example potential functions
def example_potential(phi):
    """Example potential V(phi)."""
    return phi**2 - phi  # Replace with your desired V(phi)

def example_potential_derivative(phi):
    """Derivative of the example potential V'(phi)."""
    return 2 * phi - 1  # Derivative of the above potential


# Parameters
dx = 0.1  # Grid spacing
N = 100   # Number of points

# Create the Wess-Zumino Hamiltonian
H_wz = create_wess_zumino_hamiltonian(example_potential, example_potential_derivative, dx, N)

# Display the Hamiltonian
print("Wess-Zumino Hamiltonian:")
print(H_wz)


ValueError: dimension mismatch